# **Stacking from scratch**
Stacking (Stacked Generalization) adalah metode ensemble learning yang menggabungkan beberapa model (base learners) untuk meningkatkan akurasi prediksi.

- Tiap base model dilatih pada dataset yang sama.
- Hasil prediksi dari model-model tersebut digunakan sebagai input features bagi meta model (final estimator).
- Tujuannya: agar meta model bisa mempelajari cara menggabungkan prediksi model dasar dengan lebih optimal.

Perbedaan utama dengan Blending:
- Stacking menggunakan k-fold cross-validation untuk membuat meta-data → lebih robust.
- Blending biasanya hanya memakai hold-out validation (misal 10–20% data untuk meta learner).


Source : https://www.geeksforgeeks.org/machine-learning/stacking-in-machine-learning/
Youtube : https://www.youtube.com/watch?v=a4IS1Ai7GCI

In [1]:
# Import Libraries
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.base import clone
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

**Pendefinisian class stack**

Bertujuan untuk membuat kerangka kerja ensemble learning berbasis stacking dan blending secara modular dan fleksibel. Class ini memungkinkan pengguna untuk menggabungkan beberapa model dasar (base learners) dan satu model meta (final estimator) untuk menghasilkan prediksi akhir yang lebih akurat. Dengan mengatur parameter seperti metode cross-validation, penggunaan blending, serta paralelisasi proses, class ini memberikan kontrol penuh kepada pengguna dalam menerapkan teknik stacking sesuai kebutuhan. Kegunaan utamanya adalah sebagai alat bantu untuk mengimplementasikan ensemble model dari nol (from scratch) tanpa tergantung pada fungsi otomatis dari pustaka seperti scikit-learn, sehingga cocok untuk pembelajaran konsep maupun eksperimen lanjutan dalam machine learning.

Buat class bernama Stack yang berisi <br>

Attribute:


Method:


In [2]:
# Definisikan class stack
class Stack:
    def __init__(self, base_models, meta_model, n_folds=5, blending=False, random_state=None):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
        self.blending = blending
        self.random_state = random_state

    def fit_predict(self, X_train, y_train, X_test):
        # Menyimpan prediksi base model
        meta_features_train = np.zeros((X_train.shape[0], len(self.base_models)))
        meta_features_test = np.zeros((X_test.shape[0], len(self.base_models)))

        # KFold Cross Validation
        kf = KFold(n_splits=self.n_folds, shuffle=True, random_state=self.random_state)

        for i, model in enumerate(self.base_models):
            test_fold_preds = np.zeros((X_test.shape[0], self.n_folds))
            print(f"Training base model {i+1}: {model.__class__.__name__}")

            for j, (train_idx, val_idx) in enumerate(kf.split(X_train)):
                X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
                y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

                m = clone(model)
                m.fit(X_tr, y_tr)

                # Simpan prediksi untuk meta features
                meta_features_train[val_idx, i] = m.predict(X_val)
                test_fold_preds[:, j] = m.predict(X_test)

            # Rata-rata hasil prediksi tiap fold untuk test set
            meta_features_test[:, i] = test_fold_preds.mean(axis=1)

        # Training meta model
        self.meta_model.fit(meta_features_train, y_train)
        final_pred = self.meta_model.predict(meta_features_test)

        return final_pred

### **Upload datasets**

In [3]:
# Load dataset iris dari sklearn
from sklearn.datasets import load_iris
iris = load_iris(as_frame=True)
glass_df = iris.frame

# Pisahkan fitur dan target
X1 = glass_df.drop(columns=['target'])
y1 = glass_df['target']

# Encode target (optional, untuk case multiclass stacking)
le = LabelEncoder()
y1 = le.fit_transform(y1)

# Split data menjadi train dan test
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, random_state=0)

# **Model training and evaluation of the obtained results**
Both in the case of classification and regression, stacking and blending showed the same and not the best results. As a rule, this situation occurs for two reasons: given that metadata is based on predictions of basic models, the presence of weak basic models can reduce the accuracy of stronger ones which will reduce the final prediction as a whole. Also a small amount of training data often leads to overfitting which in turn reduces the accuracy of predictions.

In this case the problem can be partially solved by setting stack_method='predict_proba' when each basic classifier outputs class membership probabilities instead of the classes themselves which can help increase accuracy in the case of non-mutually exclusive classes. Also this method works better with noise in the data. As you can see this method has significantly increased the accuracy of the model. With the right selection of models and hyperparameters the accuracy will be even higher.

Most often stacking shows slightly better results than blending due to the use of k-fold cross-validation but usually the difference is noticeable only on a large amount of data.

In [ ]:
# StackingClassifier dan Blending Classifier

**StackingClassifier (scikit-learn)**


* Merupakan implementasi resmi stacking untuk klasifikasi dalam scikit-learn.

* Mempermudah proses ensemble dengan base learners dan final estimator (meta learner) dalam satu objek.

* Sudah menangani cross-validation secara internal sehingga mengurangi risiko data leakage.

* Mendukung parameter passthrough=True jika ingin menggabungkan fitur asli dengan meta-features.

* Cocok untuk digunakan dalam pipeline dan produksi karena stabil dan teruji.

In [6]:
# StackingClassifier (scikit-learn)
from sklearn.ensemble import StackingClassifier

# sk_estimators
sk_estimators = [
    ('lr', LogisticRegression(max_iter=500)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=0)),
    ('gb', GradientBoostingClassifier(random_state=0))
]

# stack_methods
stack_methods = ['predict', 'predict_proba']

for i, method in enumerate(stack_methods):
  print(f"\nStackingClassifier dengan stack_method = '{method}'")

# Buat objek StackingClassifier
  clf = StackingClassifier(
        estimators=sk_estimators,
        final_estimator=LogisticRegression(max_iter=500),
        stack_method=method
  )

  # Fit model ke data training
  clf.fit(X1_train, y1_train)

  # Prediksi pada data test
  pred = clf.predict(X1_test)

  # Evaluasi akurasi
  acc = accuracy_score(y1_test, pred)
  print(f"Akurasi (stack_method='{method}'): {round(acc, 4)}")


StackingClassifier dengan stack_method = 'predict'
Akurasi (stack_method='predict'): 0.9737

StackingClassifier dengan stack_method = 'predict_proba'
Akurasi (stack_method='predict_proba'): 0.9737
